Loading the Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the data
df_0 = pd.read_csv(r"C:\Users\pingk\Downloads\fadhli nitip\asikrt4_3g2_clnd_blncd.csv")

In [3]:
df_0

,prov_char,sample_code,649.893,650.376,650.858,651.34,651.822,652.304,652.786,653.268,...,3998.194,3998.676,3999.158,3999.64,4000.122,tgp_name,dgp_name,fgp_name,country_name,thnoth_name
0,SNK,TH-SNK-002-2209-015,0.015745,0.015312,0.015027,0.014932,0.015066,0.015446,0.016049,0.016803,...,0.001152,0.001019,0.000941,0.000939,0.001018,Group 1,Group 1,Group 2,Thailand,Thai
1,SNK,TH-SNK-057-2312-001,0.018690,0.019058,0.019332,0.019348,0.019045,0.018497,0.017877,0.017394,...,0.001813,0.001878,0.001947,0.002002,0.002022,Group 1,Group 1,Group 2,Thailand,Thai
2,CBI,TH-CBI-002-2208-016,0.012338,0.012035,0.011907,0.012038,0.012468,0.013170,0.014048,0.014957,...,-0.000458,-0.000471,-0.000430,-0.000329,-0.000175,Group 1,Group 1,Group 1,Thailand,Thai
3,SNK,TH-SNK-002-2208-013,0.012944,0.012616,0.012494,0.012580,0.012845,0.013237,0.013695,0.014149,...,-0.001177,-0.001212,-0.001264,-0.001328,-0.001394,Group 1,Group 1,Group 2,Thailand,Thai
4,CBI,TH-CBI-028-2312-001,0.015756,0.016640,0.017271,0.017551,0.017465,0.017080,0.016510,0.015884,...,0.000048,-0.000054,-0.000162,-0.000276,-0.000398,Group 1,Group 1,Group 1,Thailand,Thai
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,BBX,ID-BBX-068-2307-031,0.013664,0.013097,0.012600,0.012208,0.011938,0.011801,0.011799,0.011914,...,0.001151,0.001114,0.001086,0.001078,0.001095,Group 3,Group 2,Group 3,Indonesia,Non-Thai
191,KBX,ID-KBX-068-2307-015,0.016273,0.016736,0.016989,0.016936,0.016548,0.015875,0.015023,0.014127,...,0.001596,0.001596,0.001615,0.001644,0.001667,Group 3,Group 2,Group 4,Indonesia,Non-Thai
192,SSX,ID-SSX-068-2307-028,0.016809,0.017187,0.017191,0.016868,0.016308,0.015622,0.014912,0.014267,...,0.000892,0.000970,0.001045,0.001107,0.001147,Group 3,Group 2,Group 3,Indonesia,Non-Thai
193,NWT,TH-NWT-045-2311-001,0.017173,0.017508,0.017160,0.016226,0.014927,0.013566,0.012419,0.011688,...,0.002272,0.002295,0.002304,0.002303,0.002299,Group 3,Group 2,Group 3,Thailand,Thai


Selecting Regions of Interest

In [4]:
# Define the regions of interest
regions_of_interest = [
    (2986, 3026), (2933, 2973), (2902, 2942), (2833, 2873),
    (1480, 2020), (1445, 1485), (1398, 1438), (1358, 1398),
    (1215, 1255), (1140, 1180), (1096, 1136), (1078, 1118),
    (1000, 910), (702, 742)
]

# Extract columns corresponding to the regions of interest
columns_to_focus = []
for start, end in regions_of_interest:
    columns_to_focus.extend([col for col in df_0.columns[4:-5] if start <= float(col) <= end])

# Create a new DataFrame with the selected regions
df_0_selected_regions = df_0[columns_to_focus]

# Combine the selected regions with the target column and other relevant columns
df_0_selected_regions = pd.concat([df_0[['tgp_name', 'prov_char']], df_0_selected_regions], axis=1)

# Save the DataFrame for further processing
df_0_selected_regions.to_csv('data/data file 2/data_1.csv', index=False)


FUNCTIONS

Baseline Correction

In [5]:
from scipy.signal import savgol_filter

# Function for baseline correction with dynamic window length
def baseline_correction(spectrum, default_window_length=15, polyorder=3):
    spectrum_length = len(spectrum)
    if spectrum_length < default_window_length:
        window_length = spectrum_length // 2 * 2 + 1  # Make window length odd and less than the size of the spectrum
    else:
        window_length = default_window_length
    baseline = savgol_filter(spectrum, window_length, polyorder, mode='nearest')
    corrected_spectrum = spectrum - baseline
    return corrected_spectrum

# Apply baseline correction
df_baseline_corrected_v0 = df_0_selected_regions.copy()
for col in columns_to_focus:
    df_baseline_corrected_v0[col] = baseline_correction(df_baseline_corrected_v0[col])

# Save the baseline corrected data
df_baseline_corrected_v0.to_csv('data/data file 2/data_1_bslcrct.csv', index=False)

SavGol Smoothing

In [6]:
# Function for Savitzky-Golay smoothing
def savitzky_golay_smoothing(spectrum, default_window_length=11, polyorder=2):
    window_length = min(default_window_length, len(spectrum) // 2 * 2 + 1)  # Make window length odd and less than or equal to the size of the spectrum
    if window_length < 3:  # Ensure window length is at least 3
        window_length = 3
    return savgol_filter(spectrum, window_length, polyorder, mode='nearest')  # Set mode to 'nearest'

# Apply smoothing
df_smoothed_v0 = df_baseline_corrected_v0.copy()
for col in columns_to_focus:
    df_smoothed_v0[col] = savitzky_golay_smoothing(df_smoothed_v0[col])

# Save the smoothed data
df_smoothed_v0.to_csv('data/data file 2/data_1_smoothed.csv', index=False)


Normalization

In [7]:
# Function for normalization (Min-Max scaling)
def min_max_normalization(spectrum):
    return (spectrum - np.min(spectrum)) / (np.max(spectrum) - np.min(spectrum))

# Apply normalization
df_normalized_v0 = df_smoothed_v0.copy()
for col in columns_to_focus:
    df_normalized_v0[col] = min_max_normalization(df_normalized_v0[col])

# Save the normalized data
df_normalized_v0.to_csv('data/data file 2/data_1_normalized.csv', index=False)


Derivatization (np.gradient)

In [8]:
# Calculate the first derivative using np.gradient
data_spectrum = df_normalized_v0.iloc[:, 2:].values
first_derivative_np = np.gradient(data_spectrum, axis=1)

# Calculate the second derivative using np.gradient
second_derivative_np = np.gradient(first_derivative_np, axis=1)

# Convert the results back to DataFrame
data_1_der_np = pd.DataFrame(first_derivative_np, columns=df_normalized_v0.columns[2:])
data_2_der_np = pd.DataFrame(second_derivative_np, columns=df_normalized_v0.columns[2:])

# Combine the first two columns from the original dataset with the np.gradient derivatives

# Extract the first two columns
first_two_columns = df_normalized_v0.iloc[:, :2]

# Combine the first two columns with the derivatives
data_1_der_combined = pd.concat([first_two_columns, data_1_der_np], axis=1)
data_2_der_combined = pd.concat([first_two_columns, data_2_der_np], axis=1)

# Export the combined data to CSV
data_1_der_combined.to_csv('data/data file 2/data_1_1_der.csv', index=False)
data_2_der_combined.to_csv('data/data file 2/data_1_2_der.csv', index=False)

Derivatization (SavGol)

In [9]:
# Extract the spectrum data
data_spectrum = df_normalized_v0.iloc[:, 2:].values

# Apply Savitzky-Golay filter for the first derivative
first_derivative_savgol = savgol_filter(data_spectrum, window_length=5, polyorder=2, deriv=1, axis=1)

# Apply Savitzky-Golay filter for the second derivative
second_derivative_savgol = savgol_filter(data_spectrum, window_length=5, polyorder=2, deriv=2, axis=1)

# Convert the results back to DataFrame
data_1_der_savgol = pd.DataFrame(first_derivative_savgol, columns=df_normalized_v0.columns[2:])
data_2_der_savgol = pd.DataFrame(second_derivative_savgol, columns=df_normalized_v0.columns[2:])

# Extract the first two columns
first_two_columns = df_normalized_v0.iloc[:, :2]

# Combine the first two columns with the Savitzky-Golay derivatives
data_1_der_savgol_combined = pd.concat([first_two_columns, data_1_der_savgol], axis=1)
data_2_der_savgol_combined = pd.concat([first_two_columns, data_2_der_savgol], axis=1)

# Export the combined data to CSV
data_1_der_savgol_combined.to_csv('data/data file 2/data_1_1_der_savgol.csv', index=False)
data_2_der_savgol_combined.to_csv('data/data file 2/data_1_2_der_savgol.csv', index=False)

SNV

In [10]:
def snv(spectrum):
    return (spectrum - np.mean(spectrum)) / np.std(spectrum)

# Apply SNV to the selected regions
df_snv = df_0_selected_regions.copy()
for col in columns_to_focus:
    df_snv[col] = snv(df_snv[col])

# Save the SNV data
df_snv.to_csv('data/data file 2/data_1_snv.csv', index=False)


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\numpy\_core\fromnumeric.py:3800: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


Random Normal Variate (RNV)

In [11]:
def rnv(spectrum):
    random_noise = np.random.normal(0, np.std(spectrum), spectrum.shape)
    return spectrum + random_noise

# Apply RNV to the selected regions
df_rnv = df_0_selected_regions.copy()
for col in columns_to_focus:
    df_rnv[col] = rnv(df_rnv[col])

# Save the RNV data
df_rnv.to_csv('data/data file 2/data_1_rnv.csv', index=False)


Multiplicative Scatter Correction (MSC)

In [12]:
# def msc(spectrum, reference):
#     mean_spectrum = np.mean(reference, axis=0)
#     fit = np.polyfit(mean_spectrum, spectrum, 1, full=True)
#     corrected_spectrum = (spectrum - fit[0][1]) / fit[0][0]
#     return corrected_spectrum

# # Apply MSC to the selected regions
# df_msc = df_0_selected_regions.copy()
# for col in columns_to_focus:
#     df_msc[col] = msc(df_msc[col])

# # Save the MSC data
# df_msc.to_csv('data/data file 2/data_1_msc2.csv', index=False)


In [13]:
# Multiplicative Scatter Correction (MSC) function
def msc(input_data):
    # Mean center the data
    mean_spectrum = np.mean(input_data, axis=0)
    input_data_centered = input_data - mean_spectrum

    # Perform MSC
    reference = np.mean(input_data, axis=0)
    msc_data = np.zeros_like(input_data)

    for i in range(input_data.shape[0]):
        fit = np.polyfit(reference, input_data_centered[i, :], 1, full=True)
        msc_data[i, :] = (input_data_centered[i, :] - fit[0][1]) / fit[0][0]
    
    return msc_data

# Extract the spectral data from the dataframe
spectral_data = df_0_selected_regions.iloc[:, 2:].values

# Apply MSC
msc_corrected_data = msc(spectral_data)

# Create a new dataframe with the MSC corrected data
msc_df = pd.DataFrame(msc_corrected_data, columns=df_0_selected_regions.columns[2:])
msc_df.insert(0, 'prov_char', df_0_selected_regions['prov_char'])
msc_df.insert(0, 'tgp_name', df_0_selected_regions['tgp_name'])

# Save the MSC corrected data to a new CSV file
msc_corrected_file_path = 'data/data file 2/data_1_msc_corrected.csv'
msc_df.to_csv(msc_corrected_file_path, index=False)


Classification and Evaluation (40-fold)

In [14]:
# Define the target variable
target = 'tgp_name'

# Ensure columns_to_focus are correctly identified
numeric_cols_df_0 = df_0_selected_regions.select_dtypes(include=[np.number]).columns.tolist()
columns_to_focus = numeric_cols_df_0  # Ensure columns are correctly selected

# Classification and evaluation function using 40-fold CV
def classify_and_evaluate(df, columns):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score
    from sklearn.preprocessing import LabelEncoder

    # Encode target variable
    le = LabelEncoder()
    y = le.fit_transform(df[target])

    # Define features
    X = df[columns]

    # Initialize the classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Perform cross-validation
    scores = cross_val_score(model, X, y, cv=40)

    # Print the results
    print(f'Cross-Validation Accuracy: {np.mean(scores)}')


In [15]:

# Evaluate each preprocessing method
print("Evaluation for No Preprocessing:")
classify_and_evaluate(df_0_selected_regions, columns_to_focus)

Evaluation for No Preprocessing:
Cross-Validation Accuracy: 0.77375


In [16]:
print("Evaluation for Baseline Correction:")
df_baseline_corrected_v0z = pd.read_csv('data/data file 2/data_1_bslcrct.csv')
classify_and_evaluate(df_baseline_corrected_v0z, columns_to_focus)

Evaluation for Baseline Correction:
Cross-Validation Accuracy: 0.5537500000000001


In [17]:
print("Evaluation for Smoothing:")
df_smoothed_v0z = pd.read_csv('data/data file 2/data_1_smoothed.csv')
classify_and_evaluate(df_smoothed_v0z, columns_to_focus)

Evaluation for Smoothing:
Cross-Validation Accuracy: 0.6787500000000001


In [18]:
print("Evaluation for Normalization:")
df_normalized_v0z = pd.read_csv('data/data file 2/data_1_normalized.csv')
classify_and_evaluate(df_normalized_v0z, columns_to_focus)

Evaluation for Normalization:
Cross-Validation Accuracy: 0.65625


In [19]:
print("Evaluation for 1-Derivative Spectroscopy:")
data_1_der_combined_v0z = pd.read_csv('data/data file 2/data_1_1_der.csv')
classify_and_evaluate(data_1_der_combined_v0z, columns_to_focus)

Evaluation for 1-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.67625


In [20]:
print("Evaluation for 2-Derivative Spectroscopy:")
data_2_der_combined_v0z = pd.read_csv('data/data file 2/data_1_2_der.csv')
classify_and_evaluate(data_2_der_combined_v0z, columns_to_focus)

Evaluation for 2-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.72875


In [21]:
print("Evaluation for 1-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v0z = pd.read_csv('data/data file 2/data_1_1_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v0z, columns_to_focus)

Evaluation for 1-SG-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.7100000000000001


In [22]:
print("Evaluation for 2-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v0z = pd.read_csv('data/data file 2/data_1_2_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v0z, columns_to_focus)

Evaluation for 2-SG-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.74


In [23]:
print("Evaluation for SNV:")
df_snv_v0z = pd.read_csv('data/data file 2/data_1_snv.csv')
classify_and_evaluate(df_snv_v0z, columns_to_focus)

print("Evaluation for RNV:")
df_rnv_v0z = pd.read_csv('data/data file 2/data_1_rnv.csv')
classify_and_evaluate(df_rnv_v0z, columns_to_focus)

print("Evaluation for MSC:")
df_msc_v0z = pd.read_csv('data/data file 2/data_1_msc_corrected.csv')
classify_and_evaluate(df_msc_v0z, columns_to_focus)

Evaluation for SNV:
Cross-Validation Accuracy: 0.76875
Evaluation for RNV:
Cross-Validation Accuracy: 0.54875
Evaluation for MSC:
Cross-Validation Accuracy: 0.7825


Classification and Evaluation (40-fold) -with extra detailed

In [24]:
# Define the target variable
target = 'tgp_name'

# Ensure columns_to_focus are correctly identified
numeric_cols_df_0 = df_0_selected_regions.select_dtypes(include=[np.number]).columns.tolist()
columns_to_focus = numeric_cols_df_0  # Ensure columns are correctly selected

# Classification and evaluation function using 40-fold CV with detailed metrics
def classify_and_evaluate(df, columns):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score, StratifiedKFold
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
    import numpy as np

    # Encode target variable
    le = LabelEncoder()
    y = le.fit_transform(df[target])

    # Define features
    X = df[columns]

    # Initialize the classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Initialize Stratified K-Fold Cross-Validation
    skf = StratifiedKFold(n_splits=40)

    # Arrays to store results
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    conf_matrices = []

    # Perform cross-validation
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Train the model
        model.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, average=None))
        recalls.append(recall_score(y_test, y_pred, average=None))
        f1_scores.append(f1_score(y_test, y_pred, average=None))
        conf_matrices.append(confusion_matrix(y_test, y_pred))

    # Calculate mean scores
    mean_accuracy = np.mean(accuracies)
    mean_precision = np.mean(precisions, axis=0)
    mean_recall = np.mean(recalls, axis=0)
    mean_f1 = np.mean(f1_scores, axis=0)
    mean_conf_matrix = np.mean(conf_matrices, axis=0)

    # Print the results
    print(f'Cross-Validation Accuracy: {mean_accuracy}')
    print(f'Precision per class: {mean_precision}')
    print(f'Recall per class: {mean_recall}')
    print(f'F1-score per class: {mean_f1}')


In [25]:
# Evaluate each preprocessing method
print("Evaluation for No Preprocessing:")
classify_and_evaluate(df_0_selected_regions, columns_to_focus)

print("Evaluation for Baseline Correction:")
df_baseline_corrected_v0y = pd.read_csv('data/data file 2/data_1_bslcrct.csv')
classify_and_evaluate(df_baseline_corrected_v0y, columns_to_focus)

print("Evaluation for Smoothing:")
df_smoothed_v0y = pd.read_csv('data/data file 2/data_1_smoothed.csv')
classify_and_evaluate(df_smoothed_v0y, columns_to_focus)

print("Evaluation for Normalization:")
df_normalized_v0y = pd.read_csv('data/data file 2/data_1_normalized.csv')
classify_and_evaluate(df_normalized_v0y, columns_to_focus)

print("Evaluation for 1-Derivative Spectroscopy:")
df_derivative_v0y = pd.read_csv('data/data file 2/data_1_1_der.csv')
classify_and_evaluate(df_derivative_v0y, columns_to_focus)

print("Evaluation for 2-Derivative Spectroscopy:")
data_2_der_combined_v0y = pd.read_csv('data/data file 2/data_1_2_der.csv')
classify_and_evaluate(data_2_der_combined_v0y, columns_to_focus)

print("Evaluation for 1-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v0y = pd.read_csv('data/data file 2/data_1_1_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v0y, columns_to_focus)

print("Evaluation for 2-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v0y = pd.read_csv('data/data file 2/data_1_2_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v0y, columns_to_focus)

Evaluation for No Preprocessing:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.77375
Precision per class: [0.825      0.71666667 0.74166667]
Recall per class: [0.8625 0.675  0.7625]
F1-score per class: [0.82       0.66416667 0.72916667]
Evaluation for Baseline Correction:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.5537500000000001
Precision per class: [0.54375    0.51458333 0.53125   ]
Recall per class: [0.55   0.5125 0.5875]
F1-score per class: [0.50833333 0.48083333 0.51916667]
Evaluation for Smoothing:


...

c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.7100000000000001
Precision per class: [0.75416667 0.69166667 0.68958333]
Recall per class: [0.75   0.675  0.8125]
F1-score per class: [0.71833333 0.63666667 0.71166667]
Evaluation for 2-SG-Derivative Spectroscopy:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.74
Precision per class: [0.775      0.71666667 0.6875    ]
Recall per class: [0.8    0.7125 0.7375]
F1-score per class: [0.75583333 0.68583333 0.68      ]


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
print("Evaluation for SNV:")
df_snv_v0y = pd.read_csv('data/data file 2/data_1_snv.csv')
classify_and_evaluate(df_snv_v0y, columns_to_focus)

print("Evaluation for RNV:")
df_rnv_v0y = pd.read_csv('data/data file 2/data_1_rnv.csv')
classify_and_evaluate(df_rnv_v0y, columns_to_focus)

print("Evaluation for MSC:")
df_msc_v0y = pd.read_csv('data/data file 2/data_1_msc_corrected.csv')
classify_and_evaluate(df_msc_v0y, columns_to_focus)

Evaluation for SNV:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.76875
Precision per class: [0.79583333 0.69166667 0.77916667]
Recall per class: [0.8625 0.675  0.7375]
F1-score per class: [0.80083333 0.65166667 0.7325    ]
Evaluation for RNV:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.54875
Precision per class: [0.475      0.44166667 0.51666667]
Recall per class: [0.5375 0.5375 0.5375]
F1-score per class: [0.48       0.45833333 0.49166667]
Evaluation for MSC:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.7825
Precision per class: [0.80833333 0.61       0.75833333]
Recall per class: [0.9    0.6125 0.775 ]
F1-score per class: [0.82833333 0.58428571 0.74083333]


Classification and Evaluation using LOGO-CV

In [27]:
# Classification and evaluation function using LOGO-CV with detailed metrics
def classify_and_evaluate_logo_cv_detailed(df, columns):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import LeaveOneGroupOut
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
    
    # Encode target variable
    le = LabelEncoder()
    y = le.fit_transform(df[target])
    
    # Define features
    X = df[columns]
    
    # Initialize the classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Initialize LOGO-CV
    logo = LeaveOneGroupOut()
    groups = df['prov_char']
    
    # Arrays to store results
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    all_y_test = []
    all_y_pred = []
    
    # Perform LOGO-CV
    for train_idx, test_idx in logo.split(X, y, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict and evaluate
        y_pred = model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        all_y_test.extend(y_test)
        all_y_pred.extend(y_pred)
        
    # Calculate overall metrics
    mean_accuracy = np.mean(accuracies)
    precision = precision_score(all_y_test, all_y_pred, average=None)
    recall = recall_score(all_y_test, all_y_pred, average=None)
    f1 = f1_score(all_y_test, all_y_pred, average=None)
    conf_matrix = confusion_matrix(all_y_test, all_y_pred)
    
    # Print the results
    print(f'Mean Accuracy: {mean_accuracy}')
    print(f'Precision per class: {precision}')
    print(f'Recall per class: {recall}')
    print(f'F1-score per class: {f1}')
    
    # Return confusion matrix for presentation
    return conf_matrix

In [28]:
# Evaluate each preprocessing method using LOGO-CV with detailed metrics
print("LOGO-CV Evaluation for No Preprocessing:")
conf_matrix_no_preprocessing_v0x = classify_and_evaluate_logo_cv_detailed(df_0_selected_regions, columns_to_focus)

print("LOGO-CV Evaluation for Baseline Correction:")
df_baseline_corrected_v0x = pd.read_csv('data/data file 2/data_1_bslcrct.csv')
conf_matrix_baseline_v0x = classify_and_evaluate_logo_cv_detailed(df_baseline_corrected_v0x, columns_to_focus)

print("LOGO-CV Evaluation for Smoothing:")
df_smoothed_v0x = pd.read_csv('data/data file 2/data_1_smoothed.csv')
conf_matrix_smoothing_v0x = classify_and_evaluate_logo_cv_detailed(df_smoothed_v0x, columns_to_focus)

print("LOGO-CV Evaluation for Normalization:")
df_normalized_v0x = pd.read_csv('data/data file 2/data_1_normalized.csv')
conf_matrix_normalization_v0x = classify_and_evaluate_logo_cv_detailed(df_normalized_v0x, columns_to_focus)

print("LOGO-CV Evaluation for 1-Derivative Spectroscopy:")
df_1_derivative_v0x = pd.read_csv('data/data file 2/data_1_1_der.csv')
conf_matrix_1_derivative_v0x = classify_and_evaluate_logo_cv_detailed(df_1_derivative_v0x, columns_to_focus)

print("LOGO-CV Evaluation for 2-Derivative Spectroscopy:")
df_2_derivative_v0x = pd.read_csv('data/data file 2/data_1_2_der.csv')
conf_matrix_2_derivative_v0x = classify_and_evaluate_logo_cv_detailed(df_2_derivative_v0x, columns_to_focus)

print("LOGO-CV Evaluation for 1-SG-Derivative Spectroscopy:")
df_1_der_savgol_combined_v0y = pd.read_csv('data/data file 2/data_1_1_der_savgol.csv')
conf_matrix_1_sg_v0x = classify_and_evaluate_logo_cv_detailed(df_1_der_savgol_combined_v0y, columns_to_focus)

print("LOGO-CV Evaluation for 2-SG-Derivative Spectroscopy:")
df_2_der_savgol_combined_v0y = pd.read_csv('data/data file 2/data_1_2_der_savgol.csv')
conf_matrix_2_sg_v0x = classify_and_evaluate_logo_cv_detailed(df_2_der_savgol_combined_v0y, columns_to_focus)

print("LOGO-CV Evaluation for SNV:")
df_snv_v0x = pd.read_csv('data/data file 2/data_1_snv.csv')
conf_matrix_snv_v0x = classify_and_evaluate_logo_cv_detailed(df_snv_v0x, columns_to_focus)

print("LOGO-CV Evaluation for RNV:")
df_rnv_v0x = pd.read_csv('data/data file 2/data_1_rnv.csv')
conf_matrix_rnv_v0x = classify_and_evaluate_logo_cv_detailed(df_rnv_v0x, columns_to_focus)

print("LOGO-CV Evaluation for MSC:")
df_msc_v0x = pd.read_csv('data/data file 2/data_1_msc_corrected.csv')
conf_matrix_msc_v0x = classify_and_evaluate_logo_cv_detailed(df_msc_v0x, columns_to_focus)

LOGO-CV Evaluation for No Preprocessing:
Mean Accuracy: 0.3447046170204065
Precision per class: [0.42307692 0.34736842 0.25      ]
Recall per class: [0.33846154 0.50769231 0.18461538]
F1-score per class: [0.37606838 0.4125     0.21238938]
LOGO-CV Evaluation for Baseline Correction:
Mean Accuracy: 0.21290594201120516
Precision per class: [0.125      0.21794872 0.18867925]
Recall per class: [0.12307692 0.26153846 0.15384615]
F1-score per class: [0.12403101 0.23776224 0.16949153]
LOGO-CV Evaluation for Smoothing:
Mean Accuracy: 0.6213398764977712
Precision per class: [0.63636364 0.44444444 0.57894737]
Recall per class: [0.64615385 0.49230769 0.50769231]
F1-score per class: [0.64122137 0.46715328 0.54098361]
LOGO-CV Evaluation for Normalization:
Mean Accuracy: 0.6402753445385023
Precision per class: [0.65079365 0.51282051 0.59259259]
Recall per class: [0.63076923 0.61538462 0.49230769]
F1-score per class: [0.640625   0.55944056 0.53781513]
LOGO-CV Evaluation for 1-Derivative Spectroscopy:


Display Confusion Matrices

In [29]:
# Function to display confusion matrix in a tabular format
def display_confusion_matrix(conf_matrix, class_labels):
    df_cm = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
    print(df_cm)

# Ensure LabelEncoder is defined and class labels are set
from sklearn.preprocessing import LabelEncoder

# Define the target variable and fit LabelEncoder
target = 'tgp_name'
le = LabelEncoder()
le.fit(df_0_selected_regions[target])
class_labels = le.classes_

In [30]:
# Display confusion matrices for each preprocessing method
print("Confusion Matrix for No Preprocessing:")
display_confusion_matrix(conf_matrix_no_preprocessing_v0x, class_labels)

print("Confusion Matrix for Baseline Correction:")
display_confusion_matrix(conf_matrix_baseline_v0x, class_labels)

print("Confusion Matrix for Smoothing:")
display_confusion_matrix(conf_matrix_smoothing_v0x, class_labels)

print("Confusion Matrix for Normalization:")
display_confusion_matrix(conf_matrix_normalization_v0x, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_1_derivative_v0x, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_2_derivative_v0x, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_1_sg_v0x, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_2_sg_v0x, class_labels)

print("Confusion Matrix for SNV:")
display_confusion_matrix(conf_matrix_snv_v0x, class_labels)

print("Confusion Matrix for RNV:")
display_confusion_matrix(conf_matrix_rnv_v0x, class_labels)

print("Confusion Matrix for MSC:")
display_confusion_matrix(conf_matrix_msc_v0x, class_labels)

Confusion Matrix for No Preprocessing:
         Group 1  Group 2  Group 3
Group 1       22       25       18
Group 2       14       33       18
Group 3       16       37       12
Confusion Matrix for Baseline Correction:
         Group 1  Group 2  Group 3
Group 1        8       33       24
Group 2       29       17       19
Group 3       27       28       10
Confusion Matrix for Smoothing:
         Group 1  Group 2  Group 3
Group 1       42       15        8
Group 2       17       32       16
Group 3        7       25       33
Confusion Matrix for Normalization:
         Group 1  Group 2  Group 3
Group 1       41       16        8
Group 2       11       40       14
Group 3       11       22       32
Confusion Matrix for Derivative Spectroscopy:
         Group 1  Group 2  Group 3
Group 1       41       11       13
Group 2       14       33       18
Group 3        6       22       37
Confusion Matrix for Derivative Spectroscopy:
         Group 1  Group 2  Group 3
Group 1       41       1

In [31]:
def snv(spectrum):
    return (spectrum - np.mean(spectrum)) / np.std(spectrum)


# Apply SNV to the selected regions
df_snv_1 = df_smoothed_v0x.copy()
for col in columns_to_focus:
    df_snv_1[col] = snv(df_snv_1[col])

# Save the SNV data
#df_snv_1.to_csv('data/data file 4/data_1_snv.csv', index=False)


In [32]:
print("Evaluation for Smooth + SNV:")
classify_and_evaluate(df_snv_1, columns_to_focus)

Evaluation for Smooth + SNV:


c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pingk\OneDrive - Chulalongkorn University\Documents\mekargit\cpocluster\env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

Cross-Validation Accuracy: 0.65625
Precision per class: [0.6875     0.59791667 0.57083333]
Recall per class: [0.7125 0.725  0.575 ]
F1-score per class: [0.675      0.62083333 0.53666667]
